In [32]:
import pandas as pd 
from sklearn.model_selection import train_test_split

In [12]:
# File paths
triplets_file = r"D:\ONE DRIVE MAHI\OneDrive\Documents\GitHub\Music-Recommender-KNN\song_dataset\triplets_file.csv"
songs_metadata_file = r"D:\ONE DRIVE MAHI\OneDrive\Documents\GitHub\Music-Recommender-KNN\song_dataset\song_data.csv"

# Load the triplets file (with header in the first row)
song_df_1 = pd.read_csv(triplets_file)
song_df_1.columns = ['user_id', 'song_id', 'listen_count']

# Load the songs metadata file
song_df_2 = pd.read_csv(songs_metadata_file)

song_df = pd.merge(song_df_1, song_df_2.drop_duplicates(['song_id']), on="song_id", how="left")

print(song_df.head())


                                    user_id             song_id  listen_count  \
0  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOAKIMP12A8C130995             1   
1  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBBMDR12A8C13253B             2   
2  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBXHDL12A81C204C0             1   
3  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBYHAJ12A6701BF1D             1   
4  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SODACBL12A8C13C273             1   

             title                        release    artist_name  year  
0         The Cove             Thicker Than Water   Jack Johnson     0  
1  Entre Dos Aguas            Flamenco Para Niños  Paco De Lucia  1976  
2         Stronger                     Graduation     Kanye West  2007  
3   Constellations              In Between Dreams   Jack Johnson  2005  
4     Learn To Fly  There Is Nothing Left To Lose   Foo Fighters  1999  


In [23]:
#Checking listening percentage
#We are checking how many users interacted with the song, not how many times they've individually listened to it.
song_grouped=song_df.groupby(['title']).agg({'listen_count':'count'}).reset_index()
#Here listen count now represents how many users have listened to that song
grouped_sum=song_grouped['listen_count'].sum() #Summing up
song_grouped['percentage']=song_grouped['listen_count'].div(grouped_sum)*100
# Sort the DataFrame temporarily without modifying the original
sorted_song_grouped = song_grouped.sort_values(['listen_count', 'title'], ascending=[False, True])

# View the sorted result
print(sorted_song_grouped.head())


                               title  listen_count  percentage
6836                   Sehr kosmisch          8277     0.41385
8725                            Undo          7032     0.35160
1964  Dog Days Are Over (Radio Edit)          6949     0.34745
9496                  You're The One          6729     0.33645
6498                         Revelry          6145     0.30725


In [ ]:
#Return unique users and unique songs
users = song_df['user_id'].unique()
len(users)


76353

In [ ]:
songs = song_df['title'].unique()
len(songs) 

9567

In [33]:
train_data, test_data = train_test_split(song_df, test_size = 0.20, random_state=0)

In [ ]:
#Define recommeneders module

In [ ]:
pm = Recommenders.popularity_recommender_py()
pm.create(train_data, 'user_id', 'song')
#user the popularity model to make some prediction
user_id = users[5]
pm.recommend(user_id)